# AI Agent Examples

In this chapter, you will explore AI agents that can decide which tools to use and remember context across turns.

| File | What it demonstrates | GitHub Link |
|------|---------------------|-------------|
| `05_ai_agent_basics_calculator_memory.json` | Agent with Calculator + Memory | [View](https://github.com/ezponda/ai-agents-course/blob/main/courses/n8n_no_code/workflows/05_ai_agent_basics_calculator_memory.json) |
| `06_ai_agent_tools_wikipedia_calculator.json` | Agent with Wikipedia + Calculator + Think | [View](https://github.com/ezponda/ai-agents-course/blob/main/courses/n8n_no_code/workflows/06_ai_agent_tools_wikipedia_calculator.json) |
| `07_ai_agent_chat_trigger_memory.json` | Chat interface with memory across turns | [View](https://github.com/ezponda/ai-agents-course/blob/main/courses/n8n_no_code/workflows/07_ai_agent_chat_trigger_memory.json) |

**Credentials needed:** OpenRouter API key (Settings → Credentials). Wikipedia and Calculator tools need no credentials.

---

## Meet the Node: AI Agent

The **AI Agent** is fundamentally different from the Basic LLM Chain you used in previous chapters.

| Property | Basic LLM Chain | AI Agent |
|----------|-----------------|----------|
| **Execution** | One prompt → one response | Loop: think → act → observe → repeat |
| **Tools** | None | Can call Calculator, Wikipedia, HTTP, etc. |
| **Decision-making** | You control the flow | Agent decides what to do next |
| **Output field** | `text` | `output` |

**Key concept:** The agent can call tools (like Calculator) and decide when it has enough information to answer.

### Sub-Nodes (Connected via Dotted Lines)

The AI Agent uses **sub-nodes** that provide capabilities:

| Sub-node | What it provides | Required? |
|----------|------------------|-----------|
| **Chat Model** | The AI "brain" for reasoning | Yes |
| **Memory** | Remembers conversation history across turns | No |
| **Tools** | Actions the agent can take (Calculator, Wikipedia, etc.) | No |

### Tools You'll Use in This Chapter

| Tool | What it does | Needs credentials? |
|------|--------------|-------------------|
| **Calculator** | Performs arithmetic accurately | No |
| **Wikipedia** | Looks up factual information | No |
| **Think** | A scratchpad for the agent to plan before acting | No |

**See Node Toolbox for full documentation on AI Agent and all tools.**

---

## Example 1: Calculator + Memory

**File:** `05_ai_agent_basics_calculator_memory.json`

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/workflows/05_ai_agent_basics_calculator_memory.json
> ```
>
> **Download:** [05_ai_agent_basics_calculator_memory.json](https://github.com/ezponda/ai-agents-course/raw/main/courses/n8n_no_code/workflows/05_ai_agent_basics_calculator_memory.json)

### What Problem This Solves

You want an assistant that can do math reliably (using a Calculator tool instead of guessing) and remember context across multiple runs (using Memory with a session ID).

### Node-by-Node Walkthrough

| Node | Type | What it does |
|------|------|-------------|
| **Manual Trigger** | Trigger | Starts the workflow |
| **Input — Agent Prompt** | Set | Creates `chatInput` and `sessionId` fields |
| **AI Agent — Calculator + Memory** | AI Agent | Receives `{{ $json.chatInput }}`, decides to use tools, outputs `output` |
| **Output — Answer** | Set | Saves `{{ $json.output }}` as `answer` |

**Sub-nodes (dotted lines to the Agent):**

| Sub-node | Type | Purpose |
|----------|------|---------|
| **OpenRouter Chat Model** | Chat Model | Provides the AI "brain" |
| **Simple Memory** | Memory | Remembers last 10 messages using `{{ $json.sessionId }}` |
| **Calculator** | Tool | Performs arithmetic when the agent calls it |

### System Message

```
You are a concise assistant.

Rules:
- If the user asks for ANY arithmetic, ALWAYS use the Calculator tool.
- Keep the final answer short (max 5 lines).
```

**Why "ALWAYS use Calculator"?** LLMs are notoriously bad at math. They often guess wrong, especially with percentages or multi-step calculations. Forcing tool use ensures accuracy.

### Default Value Pattern

The Input node uses a useful pattern for testing:

```
{{ $json.chatInput || "What is 17% tip on 46.50? Use the Calculator tool." }}
```

This means: use incoming `chatInput` if it exists, otherwise fall back to a default test value. Helpful when developing workflows.

### Memory Window

The Simple Memory sub-node has `contextWindowLength: 10`, meaning it only remembers the last 10 messages. Older messages are forgotten. This prevents token limits from being exceeded in long conversations.

### Data Flow

```
INPUT                          OUTPUT
─────                          ──────
Trigger: { }
    ↓
Input — Agent Prompt: { chatInput: "What is 17% tip on 46.50?", sessionId: "demo" }
    ↓
AI Agent (thinks → calls Calculator → reads result): { output: "A 17% tip on $46.50 is $7.91." }
    ↓
Output — Answer: { answer: "A 17% tip on $46.50 is $7.91." }
```

### What to Observe

1. Click **Input — Agent Prompt** → see `chatInput` and `sessionId`
2. Click **AI Agent** → see `output` field with the final answer
3. Run again with: `"My name is Alex. Remember it."` → then: `"What is my name?"` (memory works if `sessionId` stays the same)

---

## Example 2: Multi-Tool Agent (Wikipedia + Calculator + Think)

**File:** `06_ai_agent_tools_wikipedia_calculator.json`

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/workflows/06_ai_agent_tools_wikipedia_calculator.json
> ```
>
> **Download:** [06_ai_agent_tools_wikipedia_calculator.json](https://github.com/ezponda/ai-agents-course/raw/main/courses/n8n_no_code/workflows/06_ai_agent_tools_wikipedia_calculator.json)

### New Tool: Think

This example introduces the **Think** tool—different from other tools because it doesn't fetch external data or perform actions.

| Property | Description |
|----------|-------------|
| **Purpose** | A reasoning scratchpad where the agent writes its plan |
| **How it helps** | Explicit planning keeps the agent organized on multi-step tasks |
| **Typical usage** | Agent calls Think before calling other tools |

**Example Think output:** "I need to look up Ada Lovelace on Wikipedia first, then calculate years since 1843."

Without Think, agents sometimes jump straight to tools and get confused about the order of operations.

### What Problem This Solves

You want an agent that can look up facts (Wikipedia), do math (Calculator), and plan before acting (Think tool). The agent decides which tool to use based on the question.

### Node-by-Node Walkthrough

| Node | Type | What it does |
|------|------|-------------|
| **Manual Trigger** | Trigger | Starts the workflow |
| **Input — Agent Prompt** | Set | Creates `chatInput` and `sessionId` fields |
| **AI Agent — Multi-Tool** | AI Agent | Receives `{{ $json.chatInput }}`, chooses tools, outputs `output` |
| **Output — Answer** | Set | Saves `{{ $json.output }}` as `answer` |

**Sub-nodes (dotted lines to the Agent):**

| Sub-node | Type | Purpose |
|----------|------|---------|
| **OpenRouter Chat Model** | Chat Model | Provides the AI "brain" |
| **Simple Memory** | Memory | Remembers last 10 messages using `{{ $json.sessionId }}` |
| **Wikipedia** | Tool | Looks up factual information (no API key needed) |
| **Calculator** | Tool | Performs arithmetic |
| **Think** | Tool | A "scratchpad" where the agent writes its reasoning |

### System Message

```
You are a concise assistant.

Rules:
- For factual questions about people/places/dates: verify with Wikipedia.
- For arithmetic: ALWAYS use Calculator.
- Use Think briefly (1-2 sentences) to plan before calling tools.
- Output: short bullets, then the final computed number.
```

### Example Default Prompt

The workflow's default test question demonstrates multi-tool orchestration:

```
Using Wikipedia, tell me who Ada Lovelace was (2 bullets). 
Then give the year of her Notes and compute how many years ago that was from 2026.
```

This requires: Think (plan) → Wikipedia (facts) → Calculator (math) → format answer.

### Data Flow

```
INPUT                          OUTPUT
─────                          ──────
Trigger: { }
    ↓
Input — Agent Prompt: { chatInput: "Using Wikipedia, tell me who Ada Lovelace was...", sessionId: "demo" }
    ↓
AI Agent:
  1. Calls Think: "I'll search Wikipedia for Ada Lovelace, then calculate..."
  2. Calls Wikipedia: gets biography
  3. Calls Calculator: 2026 - 1843 = 183
  4. Formats final answer
    ↓
Output — Answer: { answer: "• Ada Lovelace was a mathematician... • 183 years ago" }
```

### What to Observe

1. Click **AI Agent — Multi-Tool** after execution
2. In the Output panel, expand the execution details to see tool calls in order
3. Look for the Think tool output—it shows the agent's planning step
4. Notice how Wikipedia is called before Calculator (the agent figured out the right order)

---

## Example 3: Chat Trigger + Memory

**File:** `07_ai_agent_chat_trigger_memory.json`

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/workflows/07_ai_agent_chat_trigger_memory.json
> ```
>
> **Download:** [07_ai_agent_chat_trigger_memory.json](https://github.com/ezponda/ai-agents-course/raw/main/courses/n8n_no_code/workflows/07_ai_agent_chat_trigger_memory.json)

### Meet the Node: Chat Trigger

This example introduces a new trigger: **Chat Trigger** (instead of Manual Trigger).

| Property | Description |
|----------|-------------|
| **Purpose** | Provides a chat interface in n8n |
| **How to use** | Click the **Chat** button (not "Execute Workflow") |
| **Outputs** | `chatInput` (user message) and `sessionId` (conversation ID) |

**Important configuration:**

| Setting | Value | Why it matters |
|---------|-------|----------------|
| `mode` | `webhook` | Listens for incoming chat messages |
| `responseMode` | `lastNode` | Returns whatever the last node outputs |

**Critical:** When `responseMode` is `lastNode`, the Chat Trigger looks for a field named **`output`** in the last node. If you use a different name (like `answer`), the chat shows raw JSON instead of the response text.

**See Node Toolbox for full documentation.**

### What Problem This Solves

You want a real chat interface (not manual test runs) where the agent remembers the conversation across multiple messages within the same session.

### Node-by-Node Walkthrough

| Node | Type | What it does |
|------|------|-------------|
| **When chat message received** | Chat Trigger | Provides chat UI, outputs `chatInput` and `sessionId` |
| **AI Agent — Chat + Memory + Calculator** | AI Agent | Receives `{{ $json.chatInput }}`, uses memory & tools, outputs `output` |
| **Output — Chat Response** | Set | Saves `{{ $json.output }}` as `output` (required name for chat) |

**Sub-nodes (dotted lines to the Agent):**

| Sub-node | Type | Purpose |
|----------|------|---------|
| **OpenRouter Chat Model** | Chat Model | Provides the AI "brain" |
| **Simple Memory (10 messages)** | Memory | Remembers last 10 messages using `{{ $json.sessionId }}` |
| **Calculator** | Tool | Performs arithmetic |

### System Message

```
You are a helpful assistant inside an n8n AI Agent workflow. 
Be concise. 
If the user asks for arithmetic, use the Calculator tool. 
If the user asks you to remember something, store it in memory and confirm briefly. 
If you don't know, say so.
```

### Why the Output Field Must Be Named `output`

This is the most common mistake with Chat Trigger workflows:

| Last node field | Chat UI shows |
|-----------------|---------------|
| `output: "Hello!"` | Hello! |
| `answer: "Hello!"` | `{"answer": "Hello!"}` (raw JSON) |
| `response: "Hello!"` | `{"response": "Hello!"}` (raw JSON) |

The Chat Trigger specifically looks for a field named `output`. That's why the Output node in this workflow saves to `output`, not `answer` like the other examples.

### Data Flow

```
INPUT (from Chat UI)           OUTPUT (to Chat UI)
────────────────────           ───────────────────
Chat Trigger: { chatInput: "My name is Alex.", sessionId: "abc123" }
    ↓
AI Agent: { output: "Got it! I'll remember your name is Alex." }
    ↓
Output — Chat Response: { output: "Got it! I'll remember your name is Alex." }
                              ↑
                     This field name matters!

(next message in same session)
Chat Trigger: { chatInput: "What is my name?", sessionId: "abc123" }
    ↓
AI Agent (reads memory): { output: "Your name is Alex." }
```

### How to Use

1. Import the workflow
2. Click **Chat** button in the n8n editor (not "Execute Workflow")
3. Type messages in the chat interface
4. Memory persists within the same chat session

### What to Observe

1. Send: `"My name is Alex. Remember it."`
2. Send: `"What is my name?"` → Agent should recall "Alex"
3. Send: `"Compute 19 * 27"` → Agent uses Calculator tool
4. Open a new chat session → Memory resets (different `sessionId`)

---

## The Agent Loop

When an AI Agent runs, it follows this pattern:

1. **Read** the question and system message
2. **Think:** "Do I need a tool?"
3. **If yes:** Call a tool, read its result
4. **Think:** "Do I have enough info now?"
5. **Repeat** until confident, then output the final answer

This loop is why agents are powerful—they can handle complex questions that require multiple steps or tools.

---

## Memory and Session IDs

Memory uses a **Session ID** to keep conversations separate:

| Session ID | Effect |
|------------|--------|
| Same `sessionId` | Same conversation thread (memory persists) |
| Different `sessionId` | Fresh start (no memory) |

In the example workflows, memory uses `{{ $json.sessionId }}` from the input. This means:
- In Manual Trigger workflows: you control the session ID in the Set node
- In Chat Trigger workflows: n8n generates a unique session ID for each chat window

---

## System Messages Control Agent Behavior

The **System Message** in the AI Agent node defines rules for the agent. Examples from the workflows:

**Workflow 05 (Calculator + Memory):**
```
You are a concise assistant.

Rules:
- If the user asks for ANY arithmetic, ALWAYS use the Calculator tool.
- Keep the final answer short (max 5 lines).
```

**Workflow 06 (Multi-Tool):**
```
You are a concise assistant.

Rules:
- For factual questions about people/places/dates: verify with Wikipedia.
- For arithmetic: ALWAYS use Calculator.
- Use Think briefly (1-2 sentences) to plan before calling tools.
```

Good system messages:
- Tell the agent **when** to use each tool
- Set **boundaries** (e.g., "max 5 lines")
- Prevent loops (e.g., "search once, then answer")

---

## Common Issues

### Agent Keeps Calling Tools Forever

**Fix:** Add boundaries to the system message:
```
Search once for current information, then provide your answer.
Do not search more than twice.
```

### Agent Never Uses Tools

**Fix:** Ask questions that clearly need external data, or make the system message more explicit about when to use tools.

### Chat Response Shows Raw JSON

**Fix:** Ensure the last node outputs a field named `output` (not `answer` or another name). The Chat Trigger expects `output`.

### Memory Doesn't Work

**Fix:** Check that `sessionId` is the same across runs. Different session IDs = different memory contexts.

---

## Summary

| Concept | What You Learned |
|---------|------------------|
| **AI Agent node** | Loops until it has enough info to answer |
| **Chat Model (sub-node)** | Provides reasoning capability |
| **Tools (sub-nodes)** | Actions the agent can call (Calculator, Wikipedia, Think) |
| **Memory (sub-node)** | Enables conversation continuity via `sessionId` |
| **System Message** | Controls when the agent uses tools and how it responds |
| **Output field** | AI Agent outputs to `output` (not `text` like Basic LLM Chain) |

**Key expression:** `{{ $json.output }}` — how you access the agent's response in the next node.

**Docs:** [AI Agent Node](https://docs.n8n.io/integrations/builtin/cluster-nodes/root-nodes/n8n-nodes-langchain.agent/)